# Collaborative Filtering

Movie Recommendation based on Non-negative Matrix Factorization.

* **Disciplines:** Unsupervised Learning, recommender systems, collaborative filtering.
* **Data:** Movies rated by users (https://grouplens.org/datasets/movielens/)

> F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. ACM Transactions on Interactive Intelligent Systems (TiiS) 5, 4: 19:1–19:19. <https://doi.org/10.1145/2827872>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm
import seaborn as sns
import os.path

In [73]:
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.decomposition import NMF

In [28]:
from fuzzywuzzy import process

/Users/raphael/opt/anaconda3/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


## Load, clean and wrangle data

In [3]:
DATA_SET_ROOT = '../data/ml-latest-small/'
WEB_APP_DATA_ROOT = './recommender/data'

In [4]:
df_movies = pd.read_csv(os.path.join(DATA_SET_ROOT,'movies.csv'), index_col='movieId')

In [5]:
df_ratings = pd.read_csv(os.path.join(DATA_SET_ROOT,'ratings.csv'))

In [6]:
df_ratings = df_ratings.merge(df_movies['title'], on='movieId')

In [7]:
# filter for movies that have at minimum N raitings
min_rating_count = 10
# https://stackoverflow.com/a/29791952
df_ratings['raiting_count_per_movie'] = df_ratings.groupby('movieId')['movieId'].transform('count')
df_ratings = df_ratings[df_ratings.raiting_count_per_movie > min_rating_count]

In [8]:
df_ratings.head()

,userId,movieId,rating,timestamp,title,raiting_count_per_movie
0,1,1,4.0,964982703,Toy Story (1995),215
1,5,1,4.0,847434962,Toy Story (1995),215
2,7,1,4.5,1106635946,Toy Story (1995),215
3,15,1,2.5,1510577970,Toy Story (1995),215
4,17,1,4.5,1305696483,Toy Story (1995),215


* *https://stackoverflow.com/a/39358924*
* *https://stackoverflow.com/q/45312377*

In [9]:
M_movie_genres = df_movies.genres.str.get_dummies().drop('(no genres listed)', axis=1)

In [10]:
M_ratings = df_ratings.pivot(columns='title', values='rating', index='userId').dropna(how='all')
M_ratings.head()

title,"'burbs, The (1989)",(500) Days of Summer (2009),10 Cloverfield Lane (2016),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),12 Angry Men (1957),12 Years a Slave (2013),127 Hours (2010),...,Zack and Miri Make a Porno (2008),Zero Dark Thirty (2012),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),Zootopia (2016),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Imputation

We have to apply imputation on the user rating matrix, because NMF cannot deal with missing values (NaN).

In [11]:
imputer = KNNImputer(n_neighbors=3)

In [12]:
R_true = imputer.fit_transform(M_ratings)

In [13]:
R_true = pd.DataFrame(data=R_true, columns=M_ratings.columns)

For the recommendation web service we will impute the user vector with the mean movie ratings.

In [14]:
generic_user_vector = M_ratings.mean(skipna=True, axis=0)

Save preprocessed data for web service.

In [15]:
R_true.to_json(os.path.join(WEB_APP_DATA_ROOT,'user_rating_matrix.json'))

In [16]:
generic_user_vector.to_json(os.path.join(WEB_APP_DATA_ROOT,'generic_user_vector.json'))
# read with pd.read_json(..., typ='series')

## NMF Recommendation

In [29]:
generic_user_vector

title
'burbs, The (1989)                   3.176471
(500) Days of Summer (2009)          3.666667
10 Cloverfield Lane (2016)           3.678571
10 Things I Hate About You (1999)    3.527778
10,000 BC (2008)                     2.705882
                                       ...   
Zoolander (2001)                     3.509259
Zootopia (2016)                      3.890625
eXistenZ (1999)                      3.863636
xXx (2002)                           2.770833
¡Three Amigos! (1986)                3.134615
Length: 2121, dtype: float64

In [31]:
generic_user_vector.index

Index([''burbs, The (1989)', '(500) Days of Summer (2009)',
       '10 Cloverfield Lane (2016)', '10 Things I Hate About You (1999)',
       '10,000 BC (2008)', '101 Dalmatians (1996)',
       '101 Dalmatians (One Hundred and One Dalmatians) (1961)',
       '12 Angry Men (1957)', '12 Years a Slave (2013)', '127 Hours (2010)',
       ...
       'Zack and Miri Make a Porno (2008)', 'Zero Dark Thirty (2012)',
       'Zero Effect (1998)', 'Zodiac (2007)', 'Zombieland (2009)',
       'Zoolander (2001)', 'Zootopia (2016)', 'eXistenZ (1999)', 'xXx (2002)',
       '¡Three Amigos! (1986)'],
      dtype='object', name='title', length=2121)

In [47]:
process.extractOne("Star Wars", generic_user_vector.index)

('Rogue One: A Star Wars Story (2016)', 90)

In [179]:
def clamp_rating(rating):
    """ clamp rating to range of [1,5] """
    return min(max(1,rating),5)

class MovieRecommenderNMF:
    def __init__(self,
                 rating_matrix_path=os.path.join(WEB_APP_DATA_ROOT,'user_rating_matrix.json'),
                 generic_user_vec_path=os.path.join(WEB_APP_DATA_ROOT,'generic_user_vector.json')):
        self.rating_matrix = pd.read_json(rating_matrix_path)
        self.generic_user_vec = pd.read_json(generic_user_vec_path, typ='series')
    
        self.model = NMF(n_components=50)
        
    def recommend(self, user_input):
        """
        user_input : dict
            Dictionary with raw movie titles as keys and raiting [1,5] as values.
            
        Returns
        -------
        List of suggested movies; input matches.
        """
        input_matches = []
        
        # create user vector
        uvec = self.generic_user_vec.copy()
        uvec[:] = 3.
        for raw_movie_title, rating in user_input.items():
            matched_title = process.extractOne(raw_movie_title, uvec.index)[0]
            input_matches.append((matched_title, rating))
            uvec[matched_title] = clamp_rating(rating)
            print("Matched",matched_title)
        
        # embed in rating matrix
        self.rating_matrix.iloc[0] = uvec
        
        # NMF
        self.rating_matrix
        W = self.model.fit_transform(self.rating_matrix)
        H = self.model.components_
        transformed_uvec = pd.Series(data=W.dot(H)[0], index=self.generic_user_vec.index)
        
        return {'recommendations':transformed_uvec.sort_values(ascending=False)[:20],
                'matches':input_matches}

In [180]:
a = MovieRecommenderNMF()

In [181]:
#b=a.recommend({'Terminator 2':5, 'Taken':10, 'xxx':5, 'star wars':0})
b=a.recommend({'pretty woman':5, 'forest gump':5, 'american beauty':5})['recommendations']

Matched Pretty Woman (1990)
Matched Forrest Gump (1994)
Matched American Beauty (1999)


/Users/raphael/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "
/Users/raphael/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:1090: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"


In [182]:
b[:20]

Forrest Gump (1994)                                 3.740818
Three Colors: Red (Trois couleurs: Rouge) (1994)    3.722600
Half Baked (1998)                                   3.670729
Thor: Ragnarok (2017)                               3.661972
Tommy Boy (1995)                                    3.658090
Tombstone (1993)                                    3.647400
Casino (1995)                                       3.642642
American Beauty (1999)                              3.617980
Black Sheep (1996)                                  3.607447
Memento (2000)                                      3.607224
Deadpool 2 (2018)                                   3.605569
Hard Candy (2005)                                   3.592802
Running Man, The (1987)                             3.589896
Avengers: Infinity War - Part I (2018)              3.588111
Dallas Buyers Club (2013)                           3.585375
Mission: Impossible - Rogue Nation (2015)           3.585245
Eat Drink Man Woman (Yin

In [183]:
b=a.recommend({'Terminator 2':5, 'Taken':10, 'xxx':5, 'star wars':0})['recommendations']
b[:20]

Matched Terminator 2: Judgment Day (1991)
Matched Taken (2008)
Matched xXx (2002)
Matched Rogue One: A Star Wars Story (2016)


/Users/raphael/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "
/Users/raphael/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:1090: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"


Three Colors: Red (Trois couleurs: Rouge) (1994)    3.766177
Terminator 2: Judgment Day (1991)                   3.736742
Taken (2008)                                        3.689357
Memento (2000)                                      3.675543
Half Baked (1998)                                   3.643440
Eat Drink Man Woman (Yin shi nan nu) (1994)         3.638907
Tommy Boy (1995)                                    3.628280
Star Trek: First Contact (1996)                     3.612772
Forrest Gump (1994)                                 3.610137
Dawn of the Dead (1978)                             3.607609
Predator (1987)                                     3.596294
Braveheart (1995)                                   3.595212
Thor: Ragnarok (2017)                               3.595194
Shrek 2 (2004)                                      3.594517
Mission: Impossible - Rogue Nation (2015)           3.587058
Hard Candy (2005)                                   3.586499
Independence Day (a.k.a.